In [1]:
from typing import List
import requests
import json
import pandas as pd
import numpy as np
import time
import sys
from collections.abc import MutableMapping,Iterable
import yaml
import datetime
import plotly.express as px
from datetime import datetime


R_ExtractDate = datetime.today().strftime("%Y-%m-%d")
frax_start_date= '2020-12-21'
frax_contract_address = "0x853d955aCEf822Db058eb8505911ED77F175b99e"

# Load Data

In [2]:
pdf_transactions = pd.read_csv("../data_raw/frax_hist_transactions.csv")
# pdf_transactions.shape # (420059, 13)
# pdf_transactions['date'].max() #'2022-08-15'
pdf_holding = pd.read_csv("../data_raw/frax_hist_holding.csv")
# pdf_holding.shape # (840118, 7)
# pdf_holding['date'].max() #Timestamp('2022-08-15 00:00:00')
pdf_address_name = pd.read_csv("../data_checks/frax_address_name_2.csv")
# pdf_address_name.shape # (25788, 2)

In [3]:
pdf_transactions.head()

,blockNumber,timeStamp,from,to,value,transactionIndex,gas,gasPrice,gasUsed,cumulativeGasUsed,value_usd,date_time,date
0,11493365,1608508752,0x0000000000000000000000000000000000000000,0x57f627f2e848f05ffc0d10c2ab45870922b3b429,1998970008000,10,2000000,50000000000,300007,1581136,0.00,2020-12-20 23:59:12,2020-12-20
1,11493365,1608508752,0x57f627f2e848f05ffc0d10c2ab45870922b3b429,0x97c4adc5d28a86f9470c70dd91dc6cc2f20d2d4d,1998970008000,10,2000000,50000000000,300007,1581136,0.00,2020-12-20 23:59:12,2020-12-20
2,11493365,1608508752,0x642a101709e48d986844656f6ec7586bed71bedd,0xdec08cb92a506b88411da9ba290f3694be223c26,44094457473705767602,29,462643,29000000000,224353,2830679,44.09,2020-12-20 23:59:12,2020-12-20
3,11493366,1608508760,0x0000000000000000000000000000000000000000,0xf97b96d0749001d65c09e21a86a2ac156be704d0,449768251800000000000,4,501112,602905595865,217194,412075,449.77,2020-12-20 23:59:20,2020-12-20
4,11493366,1608508760,0x0000000000000000000000000000000000000000,0x934adbb72e3a8cbbd4dc0934fd2bd579948eaefb,14992275060000000000000,8,160881,250000000000,106795,1059762,14992.28,2020-12-20 23:59:20,2020-12-20


In [7]:
pdf_holding.head()

,blockNumber,timeStamp,value_usd,date_time,date,address,holding
0,11493365,1608508752,0.00,2020-12-20 23:59:12,2020-12-20,0x57f627f2e848f05ffc0d10c2ab45870922b3b429,0.00
1,11493365,1608508752,0.00,2020-12-20 23:59:12,2020-12-20,0x97c4adc5d28a86f9470c70dd91dc6cc2f20d2d4d,0.00
2,11493365,1608508752,44.09,2020-12-20 23:59:12,2020-12-20,0xdec08cb92a506b88411da9ba290f3694be223c26,44.09
3,11493365,1608508752,-0.00,2020-12-20 23:59:12,2020-12-20,0x0000000000000000000000000000000000000000,0.00
4,11493365,1608508752,-0.00,2020-12-20 23:59:12,2020-12-20,0x57f627f2e848f05ffc0d10c2ab45870922b3b429,0.00


# Process contract_name data

In [3]:
# create a column 'contract' that has the binary values contract/wallet 
# (This information is not 100% valid as many contracts are not with etherscanner.io)
pdf_address_name['contract'] = np.where(pdf_address_name['name'].isna(), 'wallet','contract') 
# special ETH address
ether_spl_address = ['0x0000000000000000000000000000000000000000',
                    '0x0000000000000000000000000000000000000001',
                    '0x000000000000000000000000000000000000dead']
pdf_address_name['contract'] = np.where(pdf_address_name['address'].isin(ether_spl_address), 'contract', pdf_address_name['contract']) 

In [39]:
pdf_address_name[pdf_address_name['contract']=='contract'].head()
# shape (721, 3)
# Number of Contracts - 721

,address,name,contract
0,0x0000000000000000000000000000000000000000,NaN,contract
11,0xef2b0895f986afd7eb7939b65e2883c5e199751f,UniV3LiquidityAMO,contract
35,0xd103fef74d05fbc20b5184fe85c7187735355db3,CurveAMO_V2,contract
39,0x22c2fe05f55f81bf32310acd9a7c51c4d7b4e443,TempleTreasury,contract
148,0xcb0bc7c879bb3e9cfeb9d8efef653f33b3d242e9,FraxswapPair,contract


In [22]:
pdf_address_name.head()

,address,name,contract
0,0x0000000000000000000000000000000000000000,NaN,wallet
1,0xef90986f38279ad8653b866691ca53f6c0528aa3,NaN,wallet
2,0x767bead0df7d2418ba59752d17afc03ab81eb015,NaN,wallet
3,0x951b6d50d07c39b0f97a7bb2f5c1e96f07a093d3,NaN,wallet
4,0x6da5dc9dc866f69776933dbf481a7f6c7498de11,NaN,wallet


# EDA on Transactions

## Prepare Data

In [4]:
pdf_transactions.head(5)

,blockNumber,timeStamp,from,to,value,transactionIndex,gas,gasPrice,gasUsed,cumulativeGasUsed,value_usd,date_time,date
0,11493365,1608508752,0x0000000000000000000000000000000000000000,0x57f627f2e848f05ffc0d10c2ab45870922b3b429,1998970008000,10,2000000,50000000000,300007,1581136,0.00,2020-12-20 23:59:12,2020-12-20
1,11493365,1608508752,0x57f627f2e848f05ffc0d10c2ab45870922b3b429,0x97c4adc5d28a86f9470c70dd91dc6cc2f20d2d4d,1998970008000,10,2000000,50000000000,300007,1581136,0.00,2020-12-20 23:59:12,2020-12-20
2,11493365,1608508752,0x642a101709e48d986844656f6ec7586bed71bedd,0xdec08cb92a506b88411da9ba290f3694be223c26,44094457473705767602,29,462643,29000000000,224353,2830679,44.09,2020-12-20 23:59:12,2020-12-20
3,11493366,1608508760,0x0000000000000000000000000000000000000000,0xf97b96d0749001d65c09e21a86a2ac156be704d0,449768251800000000000,4,501112,602905595865,217194,412075,449.77,2020-12-20 23:59:20,2020-12-20
4,11493366,1608508760,0x0000000000000000000000000000000000000000,0x934adbb72e3a8cbbd4dc0934fd2bd579948eaefb,14992275060000000000000,8,160881,250000000000,106795,1059762,14992.28,2020-12-20 23:59:20,2020-12-20


In [4]:
pdf_transactions = pd.read_csv("../data_raw/frax_hist_transactions.csv")
# Generate date related 
# convert datetime from date-str
fn_ts = lambda x: datetime.strptime(x, '%Y-%m-%d')

pdf_transactions.date = pdf_transactions.date.apply(fn_ts)
# pdf_transactions['year'] = pdf_transactions['date'].dt.year
# pdf_transactions['month'] = pdf_transactions['date'].dt.month
pdf_transactions['year-month']  = pdf_transactions['date'].dt.to_period('M').astype('str')
# join wallet name
# pdf_transactions = pdf_transactions.merge(pdf_address_name, on=['address'], how='inner')

In [48]:
pdf_transactions.head()

,blockNumber,timeStamp,from,to,value,transactionIndex,gas,gasPrice,gasUsed,cumulativeGasUsed,value_usd,date_time,date,year-month
0,11493365,1608508752,0x0000000000000000000000000000000000000000,0x57f627f2e848f05ffc0d10c2ab45870922b3b429,1998970008000,10,2000000,50000000000,300007,1581136,0.00,2020-12-20 23:59:12,2020-12-20,2020-12
1,11493365,1608508752,0x57f627f2e848f05ffc0d10c2ab45870922b3b429,0x97c4adc5d28a86f9470c70dd91dc6cc2f20d2d4d,1998970008000,10,2000000,50000000000,300007,1581136,0.00,2020-12-20 23:59:12,2020-12-20,2020-12
2,11493365,1608508752,0x642a101709e48d986844656f6ec7586bed71bedd,0xdec08cb92a506b88411da9ba290f3694be223c26,44094457473705767602,29,462643,29000000000,224353,2830679,44.09,2020-12-20 23:59:12,2020-12-20,2020-12
3,11493366,1608508760,0x0000000000000000000000000000000000000000,0xf97b96d0749001d65c09e21a86a2ac156be704d0,449768251800000000000,4,501112,602905595865,217194,412075,449.77,2020-12-20 23:59:20,2020-12-20,2020-12
4,11493366,1608508760,0x0000000000000000000000000000000000000000,0x934adbb72e3a8cbbd4dc0934fd2bd579948eaefb,14992275060000000000000,8,160881,250000000000,106795,1059762,14992.28,2020-12-20 23:59:20,2020-12-20,2020-12


In [5]:
# handle from address and to address
pdf_from_address_name = pdf_address_name.rename(columns = {'address': 'from','contract':'from_contract'}).drop('name',axis=1)
pdf_to_address_name = pdf_address_name.rename(columns = {'address': 'to','contract':'to_contract'}).drop('name',axis=1)

# join based on from address and to address
pdf_transactions_wallet = (pdf_transactions.merge(pdf_from_address_name, on='from',how='inner')
                                          .merge(pdf_to_address_name, on='to',how='inner'))
# indicates if either from or to contract is a 'contract'                                          
pdf_transactions_wallet['is_contract'] = np.where(((pdf_transactions_wallet['from_contract']=='contract')
                                                  | (pdf_transactions_wallet['to_contract']=='contract')), 'contract', 'wallet') 
pdf_transactions_wallet.head()

,blockNumber,timeStamp,from,to,value,transactionIndex,gas,gasPrice,gasUsed,cumulativeGasUsed,value_usd,date_time,date,year-month,from_contract,to_contract,is_contract
0,11493365,1608508752,0x0000000000000000000000000000000000000000,0x57f627f2e848f05ffc0d10c2ab45870922b3b429,1998970008000,10,2000000,50000000000,300007,1581136,0.00,2020-12-20 23:59:12,2020-12-20,2020-12,contract,wallet,contract
1,11493512,1608510718,0x0000000000000000000000000000000000000000,0x57f627f2e848f05ffc0d10c2ab45870922b3b429,1998970008000000000000,190,2000000,73000000000,284748,8464297,1998.97,2020-12-21 00:31:58,2020-12-21,2020-12,contract,wallet,contract
2,11493559,1608511382,0x0000000000000000000000000000000000000000,0x57f627f2e848f05ffc0d10c2ab45870922b3b429,9000008722287899316000,188,2000000,75000000000,273662,9192505,9000.01,2020-12-21 00:43:02,2020-12-21,2020-12,contract,wallet,contract
3,11493366,1608508760,0x0000000000000000000000000000000000000000,0xf97b96d0749001d65c09e21a86a2ac156be704d0,449768251800000000000,4,501112,602905595865,217194,412075,449.77,2020-12-20 23:59:20,2020-12-20,2020-12,contract,wallet,contract
4,11493434,1608509678,0x0000000000000000000000000000000000000000,0xf97b96d0749001d65c09e21a86a2ac156be704d0,4997425020000000000000,2,368907,142386609409,294823,465219,4997.43,2020-12-21 00:14:38,2020-12-21,2020-12,contract,wallet,contract


## Calculate stats - year-month level

In [11]:
pdf_txn_monthly_stats = pdf_transactions.groupby(['year-month'])['value_usd'].describe().reset_index()
# calcualte volume = sum of amnt of transactions
pdf_txn_monthly_volume = pdf_transactions.groupby(['year-month'],as_index=False)['value_usd'].agg({'volume':"sum"})
pdf_txn_monthly_stats = pdf_txn_monthly_stats.merge(pdf_txn_monthly_volume,on=['year-month'], how='inner')
pdf_txn_monthly_stats

,year-month,count,mean,std,min,25%,50%,75%,max,volume
0,2020-12,11683.0,28425.124067,1.201315e+05,0.0,1740.9700,5957.440,20983.9450,3.496231e+06,3.320907e+08
1,2021-01,24855.0,42725.077410,1.326866e+05,0.0,5000.0000,20000.000,43626.7300,4.843302e+06,1.061932e+09
2,2021-02,9768.0,52014.866203,2.821374e+05,0.0,7982.7700,18902.620,48612.2500,1.160049e+07,5.080812e+08
3,2021-03,8443.0,57192.743631,2.166769e+05,0.0,9145.9250,25693.150,50046.4500,5.763688e+06,4.828783e+08
4,2021-04,6369.0,49710.832699,1.581645e+05,0.0,6900.0100,25390.930,50000.0000,5.797101e+06,3.166083e+08
5,2021-05,10210.0,79778.217738,4.694572e+05,0.0,13646.8000,30346.560,52888.6600,1.506909e+07,8.145356e+08
6,2021-06,11324.0,51534.301459,4.101913e+05,0.0,3062.2025,13636.820,25437.6475,1.200000e+07,5.835744e+08
7,2021-07,14983.0,45400.679326,3.147604e+05,0.0,2619.6400,12241.490,26158.6900,1.400000e+07,6.802384e+08
8,2021-08,13352.0,73478.295365,5.847869e+05,0.0,3000.0000,11537.005,30080.0100,1.922142e+07,9.810822e+08
9,2021-09,10593.0,61407.502258,2.884752e+05,0.0,4397.0800,13940.980,38209.2000,1.000000e+07,6.504897e+08


In [19]:
fig = px.line(pdf_txn_monthly_stats, x='year-month', y=["count","mean","25%","50%","75%"],
title='Transaction stats across different year-months',width=1000,height=500)
fig.show()

In [20]:
fig = px.line(pdf_txn_monthly_stats, x='year-month', y=["volume"],
    title='Transaction volume across different year-months',width=1000,height=500)
fig.show()

In [53]:
fig = px.bar(pdf_txn_monthly_stats, x='year-month', y=["count","mean","min","25%","50%","75%"], barmode='group')
fig.show()

In [54]:
fig = px.bar(pdf_txn_monthly_stats, x='year-month', y=["max"], barmode='group')
fig.show()

In [55]:
fig = px.bar(pdf_txn_monthly_stats, x='year-month', y=["std"], barmode='group')
fig.show()

## Calculate  stats - contract, year-month level

In [25]:
# either to or from or both are contracts
pdf_txn_wallet_monthly_stats = pdf_transactions_wallet.groupby(['is_contract','year-month'])['value_usd'].describe().reset_index()
# calcualte volume = sum of amnt of transactions
pdf_txn_wallet_monthly_volume = pdf_transactions_wallet.groupby(['is_contract','year-month'],as_index=False)['value_usd'].agg({'volume':"sum"})
pdf_txn_wallet_monthly_stats = pdf_txn_wallet_monthly_stats.merge(pdf_txn_wallet_monthly_volume, on=['is_contract','year-month'], how='inner')
pdf_txn_wallet_monthly_stats

,is_contract,year-month,count,mean,std,min,25%,50%,75%,max,volume
0,contract,2020-12,11315.0,28295.870722,1.216633e+05,0.0,1712.0400,5793.870,20109.8600,3.496231e+06,3.201678e+08
1,contract,2021-01,22427.0,42689.988772,1.374849e+05,0.0,4921.0000,18462.380,41880.0650,4.843302e+06,9.574084e+08
2,contract,2021-02,9324.0,51353.003557,2.876491e+05,0.0,8326.9500,19145.850,47277.4925,1.160049e+07,4.788154e+08
3,contract,2021-03,8034.0,56402.198598,2.201883e+05,0.0,9501.4350,25908.590,50000.0000,5.763688e+06,4.531353e+08
4,contract,2021-04,5828.0,48711.686803,1.606950e+05,0.0,6806.4800,24566.925,49504.9500,5.797101e+06,2.838917e+08
5,contract,2021-05,8145.0,87094.591285,5.241173e+05,0.0,11922.5100,29233.900,52000.8300,1.506909e+07,7.093854e+08
6,contract,2021-06,9806.0,52759.194712,4.293498e+05,0.0,2583.1300,12627.340,25184.5075,1.200000e+07,5.173567e+08
7,contract,2021-07,13864.0,46041.864829,3.259817e+05,0.0,2511.6525,12038.565,26056.1150,1.400000e+07,6.383244e+08
8,contract,2021-08,12301.0,74398.606187,6.075419e+05,0.0,3000.0000,11164.220,29952.5700,1.922142e+07,9.151773e+08
9,contract,2021-09,9527.0,62354.071481,3.017367e+05,0.0,4618.7100,13835.400,37226.0550,1.000000e+07,5.940472e+08


In [31]:
# plot count
fig = px.line(pdf_txn_wallet_monthly_stats, x='year-month', y="count", color = "is_contract",
    title='Transaction count across different year-months for different is_contract', width=1000,height=500)
fig.show()

In [32]:
# plot mean
fig = px.line(pdf_txn_wallet_monthly_stats, x='year-month', y="mean", color = "is_contract",
    title='Transaction mean across different year-months for different is_contract',width=1000,height=500)
fig.show()

In [33]:
# plot count
fig = px.line(pdf_txn_wallet_monthly_stats, x='year-month', y="75%", color = "is_contract",
    title='Transaction value - 75 percentile across different year-months for different is_contract',width=1000,height=500)
fig.show()

In [34]:
# fig = px.bar(pdf_txn_wallet_monthly_stats, x='year-month', y=["count","mean","min","25%","50%","75%"],color="is_contract", barmode='group')
# fig.show()

## Calculate stats - from_contract, year-month level

In [35]:
# either to or from or both are contracts
pdf_txn_from_contract_monthly_stats = pdf_transactions_wallet.groupby(['from_contract','year-month'])['value_usd'].describe().reset_index()
# calcualte volume = sum of amnt of transactions
pdf_txn_from_contract_monthly_volume = pdf_transactions_wallet.groupby(['from_contract','year-month'],as_index=False)['value_usd'].agg({'volume':"sum"})
pdf_txn_from_contract_monthly_stats = pdf_txn_from_contract_monthly_stats.merge(pdf_txn_from_contract_monthly_volume, on=['from_contract','year-month'], how='inner')
pdf_txn_from_contract_monthly_stats

,from_contract,year-month,count,mean,std,min,25%,50%,75%,max,volume
0,contract,2020-12,6541.0,25729.951405,1.126351e+05,0.0,1487.5400,4991.920,17849.5300,3.496109e+06,1.682996e+08
1,contract,2021-01,13431.0,38989.443482,1.307574e+05,0.0,4224.1050,14895.290,36792.2300,4.300515e+06,5.236672e+08
2,contract,2021-02,5702.0,55419.224865,3.604678e+05,0.0,7578.9675,16098.230,42308.0125,1.160049e+07,3.160004e+08
3,contract,2021-03,4581.0,60736.863239,2.755601e+05,0.0,8424.6500,22646.140,49758.7400,5.763688e+06,2.782356e+08
4,contract,2021-04,3678.0,49109.832162,1.909410e+05,0.0,5404.7200,21617.265,48597.0550,5.797101e+06,1.806260e+08
5,contract,2021-05,5098.0,101610.164223,6.482208e+05,0.0,9289.0800,26722.595,51039.7775,1.506909e+07,5.180086e+08
6,contract,2021-06,6766.0,60131.281944,5.026145e+05,0.0,2420.3325,11429.350,24580.7800,1.200000e+07,4.068483e+08
7,contract,2021-07,9443.0,49826.863461,3.891707e+05,0.0,2336.3500,11056.550,24310.3850,1.400000e+07,4.705151e+08
8,contract,2021-08,8599.0,76259.590047,6.697102e+05,0.0,2650.9600,9994.120,27254.4850,1.922142e+07,6.557562e+08
9,contract,2021-09,6562.0,61730.831911,3.480588e+05,0.0,4140.6525,12692.440,33959.0000,1.000000e+07,4.050777e+08


In [38]:
# plot count
fig = px.line(pdf_txn_from_contract_monthly_stats, x='year-month', y="count", color = "from_contract",
    title='Transaction count across different year-months for different from_contract', width=1000,height=500)
fig.show()


In [39]:
# plot mean
fig = px.line(pdf_txn_from_contract_monthly_stats, x='year-month', y="mean", color = "from_contract",
    title='Transaction mean across different year-months for different from_contract',width=1000,height=500)
fig.show()

In [40]:
# plot 75%
fig = px.line(pdf_txn_from_contract_monthly_stats, x='year-month', y="75%", color = "from_contract",
    title='Transaction value - 75 percentile across different year-months for different from_contract',width=1000,height=500)
fig.show()

## Calculate stats - to_contract, year-month level

In [41]:
# fig = px.bar(pdf_txn_from_contract_monthly_stats, x='year-month', y=["count","mean","min","25%","50%","75%"], color="from_contract", barmode='group')
# fig.show()

In [42]:
# either to or from or both are contracts
pdf_txn_to_contract_monthly_stats = pdf_transactions_wallet.groupby(['to_contract','year-month'])['value_usd'].describe().reset_index()
# calcualte volume = sum of amnt of transactions
pdf_txn_to_contract_monthly_volume = pdf_transactions_wallet.groupby(['to_contract','year-month'],as_index=False)['value_usd'].agg({'volume':"sum"})
pdf_txn_to_contract_monthly_stats = pdf_txn_to_contract_monthly_stats.merge(pdf_txn_to_contract_monthly_volume, on=['to_contract','year-month'], how='inner')
pdf_txn_to_contract_monthly_stats

,to_contract,year-month,count,mean,std,min,25%,50%,75%,max,volume
0,contract,2020-12,6552.0,25568.483045,1.146168e+05,0.0,1595.7100,5209.510,19524.3750,3.496231e+06,1.675247e+08
1,contract,2021-01,13097.0,39891.152443,1.312113e+05,0.0,4335.3500,15374.240,38940.8400,4.843302e+06,5.224544e+08
2,contract,2021-02,5758.0,54971.154832,3.577675e+05,0.0,8002.2175,17569.760,42608.6400,1.160049e+07,3.165239e+08
3,contract,2021-03,4696.0,59093.640794,2.699137e+05,0.0,8889.8575,24822.795,49826.3925,5.763688e+06,2.775037e+08
4,contract,2021-04,3514.0,50565.047618,1.948869e+05,0.0,6900.4675,23815.040,47238.9350,5.797101e+06,1.776856e+08
5,contract,2021-05,5020.0,103397.541476,6.537045e+05,0.0,9060.7525,26234.625,50000.0000,1.506909e+07,5.190557e+08
6,contract,2021-06,6740.0,60445.946539,5.029321e+05,0.0,2352.4925,11453.100,24000.7850,1.200000e+07,4.074057e+08
7,contract,2021-07,9266.0,50478.332329,3.913780e+05,0.0,2600.5825,11219.120,24905.1700,1.400000e+07,4.677322e+08
8,contract,2021-08,8010.0,80610.608070,6.928588e+05,0.0,3399.7825,11087.620,27860.7000,1.922142e+07,6.456910e+08
9,contract,2021-09,5845.0,68823.559052,3.670456e+05,0.0,5856.5300,14979.980,36210.7300,1.000000e+07,4.022737e+08


In [44]:
# plot count
fig = px.line(pdf_txn_to_contract_monthly_stats, x='year-month', y="count", color = "to_contract",
    title='Transaction count across different year-months for different to_contract', width=1000,height=500)
fig.show()


In [45]:
# plot mean
fig = px.line(pdf_txn_to_contract_monthly_stats, x='year-month', y="mean", color = "to_contract",
    title='Transaction mean across different year-months for different to_contract', width=1000,height=500)
fig.show()


In [46]:
# plot 75%
fig = px.line(pdf_txn_to_contract_monthly_stats, x='year-month', y="75%", color = "to_contract",
    title='Transaction 75% across different year-months for different to_contract', width=1000,height=500)
fig.show()


In [47]:
# fig = px.bar(pdf_txn_to_contract_monthly_stats, x='year-month', y=["count","mean","min","25%","50%","75%"], color="to_contract", barmode='group')
# fig.show()

## Transactions Diversity

In [69]:
pdf_transactions_wallet.head()

,blockNumber,timeStamp,from,to,value,transactionIndex,gas,gasPrice,gasUsed,cumulativeGasUsed,value_usd,date_time,date,year-month,from_contract,to_contract,is_contract
0,11493365,1608508752,0x0000000000000000000000000000000000000000,0x57f627f2e848f05ffc0d10c2ab45870922b3b429,1998970008000,10,2000000,50000000000,300007,1581136,0.00,2020-12-20 23:59:12,2020-12-20,2020-12,contract,wallet,contract
1,11493512,1608510718,0x0000000000000000000000000000000000000000,0x57f627f2e848f05ffc0d10c2ab45870922b3b429,1998970008000000000000,190,2000000,73000000000,284748,8464297,1998.97,2020-12-21 00:31:58,2020-12-21,2020-12,contract,wallet,contract
2,11493559,1608511382,0x0000000000000000000000000000000000000000,0x57f627f2e848f05ffc0d10c2ab45870922b3b429,9000008722287899316000,188,2000000,75000000000,273662,9192505,9000.01,2020-12-21 00:43:02,2020-12-21,2020-12,contract,wallet,contract
3,11493366,1608508760,0x0000000000000000000000000000000000000000,0xf97b96d0749001d65c09e21a86a2ac156be704d0,449768251800000000000,4,501112,602905595865,217194,412075,449.77,2020-12-20 23:59:20,2020-12-20,2020-12,contract,wallet,contract
4,11493434,1608509678,0x0000000000000000000000000000000000000000,0xf97b96d0749001d65c09e21a86a2ac156be704d0,4997425020000000000000,2,368907,142386609409,294823,465219,4997.43,2020-12-21 00:14:38,2020-12-21,2020-12,contract,wallet,contract


In [54]:
# Txn diversity of contract-wallets 
pdf_txn_wallet_diversity_stats = pdf_transactions_wallet.groupby(['from_contract','to_contract'])['value_usd'].describe().reset_index()
pdf_txn_wallet_diversity_vol = pdf_transactions_wallet.groupby(['from_contract','to_contract'],as_index=False)['value_usd'].agg({'volume':"sum"})
pdf_txn_wallet_diversity_stats = pdf_txn_wallet_diversity_stats.merge(pdf_txn_wallet_diversity_vol, on=['from_contract','to_contract'], how='inner')
pdf_txn_wallet_diversity_stats['from_to_contract'] = pdf_txn_wallet_diversity_stats['from_contract']+"-"+pdf_txn_wallet_diversity_stats['to_contract']
pdf_txn_wallet_diversity_stats

,from_contract,to_contract,count,mean,std,min,25%,50%,75%,max,volume,from_to_contract
0,contract,contract,74795.0,252727.186416,3.153549e+06,0.0,3404.075,11619.650,35438.0700,1.511885e+08,1.890273e+10,contract-contract
1,contract,wallet,145722.0,71892.997397,4.802568e+05,0.0,3515.890,14374.755,41143.6975,7.343135e+07,1.047639e+10,contract-wallet
2,wallet,contract,139789.0,74238.246362,4.717649e+05,0.0,4717.570,16284.000,42336.3400,7.000000e+07,1.037769e+10,wallet-contract
3,wallet,wallet,59753.0,78666.802106,4.311709e+05,0.0,3204.510,14150.430,47013.7200,3.392055e+07,4.700577e+09,wallet-wallet


In [60]:
fig = px.line(pdf_txn_wallet_diversity_stats, x='from_to_contract', y=["count","mean","75%"],
    title='Transaction diversity stats', width=1000,height=500) #max])
fig.show()

In [64]:
fig = px.line(pdf_txn_wallet_diversity_stats, x='from_to_contract', y=["std"],
    title='Transaction diversity std', width=1000,height=500) #max])
fig.show()

In [65]:
fig = px.line(pdf_txn_wallet_diversity_stats, x='from_to_contract', y=["volume"],
    title='Transaction diversity volume', width=1000,height=500) #max])
fig.show()

In [75]:
fig = px.pie(pdf_txn_wallet_diversity_stats, values='volume', names='from_to_contract', 
title='Transaction diversity by volume piechart', width=500,height=500)
fig.show()

In [89]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# assign the df
df = pdf_txn_wallet_diversity_stats

# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1, cols=3, specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=df.from_to_contract, 
                    values=df.volume, name="volume"),
              1, 1)
fig.add_trace(go.Pie(labels=df.from_to_contract, 
                    values=df["count"], name="count"),
              1, 2)
fig.add_trace(go.Pie(labels=df.from_to_contract, 
      values=df["mean"], name="mean"),
              1, 3)
# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=0.4,hoverinfo="label+percent+name")

fig.update_layout(
    title_text="Transaction diversity piechart",
#     Add annotations in the center of the donut pies.
    annotations=[dict(text='volume', x=0.11, y=0.5, font_size=20, showarrow=False),
                 dict(text='count', x=0.50, y=0.5, font_size=20, showarrow=False),
                 dict(text='mean', x=0.88, y=0.5, font_size=20, showarrow=False)])
fig.show()

In [66]:
# Txn diversity of contract-wallets 
pdf_txn_wallet_diversity_month_stats = pdf_transactions_wallet.groupby(["year-month",'from_contract','to_contract'])['value_usd'].describe().reset_index()
pdf_txn_wallet_diversity_month_vol = pdf_transactions_wallet.groupby(["year-month",'from_contract','to_contract'],as_index=False)['value_usd'].agg({'volume':"sum"})
pdf_txn_wallet_diversity_month_stats = pdf_txn_wallet_diversity_month_stats.merge(pdf_txn_wallet_diversity_month_vol, on=["year-month",'from_contract','to_contract'], how='inner')
pdf_txn_wallet_diversity_month_stats['from_to_contract'] = pdf_txn_wallet_diversity_month_stats['from_contract']+"-"+pdf_txn_wallet_diversity_month_stats['to_contract']
pdf_txn_wallet_diversity_month_stats

,year-month,from_contract,to_contract,count,mean,std,min,25%,50%,75%,max,volume,from_to_contract
0,2020-12,contract,contract,1778.0,8805.700692,23465.524832,0.00,958.3975,2810.695,7602.270,420896.55,1.565654e+07,contract-contract
1,2020-12,contract,wallet,4763.0,32047.675060,130656.732773,0.00,1925.7200,6329.740,23741.245,3496109.36,1.526431e+08,contract-wallet
2,2020-12,wallet,contract,4774.0,31811.513423,132973.837128,0.00,2061.2800,7494.855,25000.000,3496231.46,1.518682e+08,wallet-contract
3,2020-12,wallet,wallet,368.0,32399.313207,55107.533069,0.00,3959.0800,19927.830,48656.440,877189.15,1.192295e+07,wallet-wallet
4,2021-01,contract,contract,4101.0,21632.104550,85046.231020,0.00,2845.1800,6495.310,25685.400,2074024.79,8.871326e+07,contract-contract
...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,2022-07,wallet,wallet,6277.0,46008.630655,219387.105852,0.00,2232.3100,5307.070,20005.490,7971155.00,2.887962e+08,wallet-wallet
80,2022-08,contract,contract,1744.0,119759.122305,913966.595061,0.00,598.6125,2963.245,20796.080,20000000.00,2.088599e+08,contract-contract
81,2022-08,contract,wallet,3787.0,58029.722458,338584.675417,0.00,2163.5700,8609.800,21772.780,10688897.28,2.197586e+08,contract-wallet
82,2022-08,wallet,contract,3818.0,61191.523853,512581.601977,0.01,2294.9300,8640.470,22644.920,20865382.41,2.336292e+08,wallet-contract


In [70]:
fig = px.line(pdf_txn_wallet_diversity_month_stats, x='year-month', y="mean",color='from_to_contract',
    title='Transaction diversity mean across different months', width=1000,height=500) #max])
fig.show()
#  color=['from_to_contract'],

In [71]:
fig = px.line(pdf_txn_wallet_diversity_month_stats, x='year-month', y="count",color='from_to_contract',
    title='Transaction diversity count across different months', width=1000,height=500) #max])
fig.show()
#  color=['from_to_contract'],

In [69]:
fig = px.line(pdf_txn_wallet_diversity_month_stats, x='year-month', y="volume",color='from_to_contract',
    title='Transaction diversity volume across different months', width=1000,height=500) #max])
fig.show()
#  color=['from_to_contract'],

### Number of unique wallets in from ,to address

In [67]:
# unique wallets which are present in from address and to address
pdf_transactions_wallet['from'].nunique(), pdf_transactions_wallet['to'].nunique()
# (24373, 25686)
# Not all address are present on both from and to (analyze more)

(24373, 25686)

In [6]:
# to-address diversity, count unique
pdf_txn_to_address_diversity = pdf_transactions_wallet.groupby(['from','from_contract','to_contract'], as_index= False)['to']\
                                                      .agg({'unique_txns': "nunique",
                                                            'total_txns': "count"})
pdf_txn_to_address_diversity['from_to_contract'] = pdf_txn_to_address_diversity['from_contract']+"-"+pdf_txn_to_address_diversity['to_contract']
pdf_txn_to_address_diversity.head(10)

,from,from_contract,to_contract,unique_txns,total_txns,from_to_contract
0,0x0000000000000000000000000000000000000000,contract,contract,48,511,contract-contract
1,0x0000000000000000000000000000000000000000,contract,wallet,781,2244,contract-wallet
2,0x0000000000000eb4ec62758aae93400b3e5f7f18,wallet,contract,4,35,wallet-contract
3,0x0000000000000eb4ec62758aae93400b3e5f7f18,wallet,wallet,6,9,wallet-wallet
4,0x00000000000017c75025d397b91d284bbe8fc7f2,wallet,contract,4,49,wallet-contract
5,0x0000000000002d534ff79e9c69e7fcc742f0be83,wallet,contract,4,267,wallet-contract
6,0x0000000000002d534ff79e9c69e7fcc742f0be83,wallet,wallet,1,1,wallet-wallet
7,0x0000000000005117dd3a72e64a705198753fdd54,wallet,contract,13,303,wallet-contract
8,0x0000000000005117dd3a72e64a705198753fdd54,wallet,wallet,8,56,wallet-wallet
9,0x000000000000660def84e69995117c0176ba446e,wallet,contract,1,2,wallet-contract


In [7]:
# sample_1 where address are sending to 1 unique address multiple times
pdf_txn_to_address_diversity_sample1 = pdf_txn_to_address_diversity[(pdf_txn_to_address_diversity['unique_txns']==1) 
                                                                    & (pdf_txn_to_address_diversity['total_txns']>1)
                                                                    & (pdf_txn_to_address_diversity['from_to_contract']=='wallet-wallet')]\
                                                            .sort_values('total_txns', ascending=False)
pdf_txn_to_address_diversity_sample1.head(10)
# pdf_txn_to_address_diversity_sample1.shape #(1178, 7)

,from,from_contract,to_contract,unique_txns,total_txns,from_to_contract
26684,0xd7c09e006a2891880331b0f6224071c1e890a98a,wallet,wallet,1,157,wallet-wallet
7793,0x3d8793145fbeb2496a8fc33346da70b6ad2572dd,wallet,wallet,1,95,wallet-wallet
7178,0x384bfb7db2e58c02dd68c8d584ba639fe4308704,wallet,wallet,1,94,wallet-wallet
29174,0xec001d0000004536cad29291f4000000d029abb2,wallet,wallet,1,73,wallet-wallet
15710,0x7d6a729a3f8055198696f5ac7587b4e086033151,wallet,wallet,1,60,wallet-wallet
3313,0x1980918297492773cb13db3fc9ad0f7233195acf,wallet,wallet,1,45,wallet-wallet
12827,0x6661b55c84ee1000edcbe4dd9cc833e26d16210c,wallet,wallet,1,39,wallet-wallet
25044,0xca436e14855323927d6e6264470ded36455fc8bd,wallet,wallet,1,35,wallet-wallet
30964,0xfa465f3bb5b9f31ea02c517122a210ddbdd413f6,wallet,wallet,1,29,wallet-wallet
13,0x00000000000080c886232e9b7ebbfb942b5987aa,wallet,wallet,1,28,wallet-wallet


In [25]:
# pdf_txn_to_address_diversity.shape #(31650, 6)

fig = px.histogram(pdf_txn_to_address_diversity,#[pdf_txn_to_address_diversity['unique_txns'].between(0,2000)],
 x="unique_txns", nbins=5000, range_x=[1,15], histnorm='percent',barmode="group", text_auto=True, color='from_to_contract',
 title= "Distribution of number of unique transactions")
#  marginal="violin") # can be `box`, `violin`)#,'to_contract'])
fig.show()
# 

In [15]:
bins = [0, 1, 2, 4, 6, 8, 10, 15, np.inf] # bins are like (0,1],(1,2]
# labels =['1','2','3','4','5','6','7','8']
labels_int = [1,2,3,4,5,6,7,8]
pdf_txn_to_address_diversity['unique_txns_bin'] = pd.cut(pdf_txn_to_address_diversity['unique_txns'], bins=bins, labels=labels_int)
# get the distribution(count) of values present in unique_txns column, count of different values of unique_exns for diffent bins/segments
pdf_txn_diversity_unq_txn_hist = pdf_txn_to_address_diversity.groupby(['from_to_contract','unique_txns_bin'], as_index=False)['unique_txns']\
                                                             .agg({'hist_unique_txns':'count'})

# for those bins, get the average value of count, count= 
pdf_txn_diversity_unq_txn_count_stats = pdf_txn_to_address_diversity.groupby(['from_to_contract', 'unique_txns_bin'], as_index=False)['total_txns']\
                                                                  .agg({'feaquency_mean':'mean',
                                                                  'feaquency_max':'max',
                                                                  'feaquency_std':'std'})
pdf_txn_address_diversity = pdf_txn_diversity_unq_txn_hist.merge(pdf_txn_diversity_unq_txn_count_stats,on=['from_to_contract', 'unique_txns_bin'], how='inner')
# pdf_txn_diversity_unq_txn_hist.shape #(32, 3)
# pdf_txn_address_diversity.shape #(32, 4)

In [14]:
pdf_txn_to_address_diversity.head()

,from,from_contract,to_contract,unique_txns,total_txns,from_to_contract,unique_txns_bin
0,0x0000000000000000000000000000000000000000,contract,contract,48,511,contract-contract,8
1,0x0000000000000000000000000000000000000000,contract,wallet,781,2244,contract-wallet,8
2,0x0000000000000eb4ec62758aae93400b3e5f7f18,wallet,contract,4,35,wallet-contract,3
3,0x0000000000000eb4ec62758aae93400b3e5f7f18,wallet,wallet,6,9,wallet-wallet,4
4,0x00000000000017c75025d397b91d284bbe8fc7f2,wallet,contract,4,49,wallet-contract,3


In [113]:
pdf_txn_to_address_diversity[(pdf_txn_to_address_diversity['unique_txns_bin']==1)
                            & (pdf_txn_to_address_diversity['from_to_contract']=='wallet-wallet')].sort_values('total_txns',ascending=False).head()

# pdf_txn_to_address_diversity[(pdf_txn_to_address_diversity['unique_txns_bin']==1)
#                             & (pdf_txn_to_address_diversity['total_txns']>1)
#                             & (pdf_txn_to_address_diversity['from_to_contract']=='wallet-wallet')].shape
# (1178, 7)

,from,from_contract,to_contract,unique_txns,total_txns,from_to_contract,unique_txns_bin
26684,0xd7c09e006a2891880331b0f6224071c1e890a98a,wallet,wallet,1,157,wallet-wallet,1
7793,0x3d8793145fbeb2496a8fc33346da70b6ad2572dd,wallet,wallet,1,95,wallet-wallet,1
7178,0x384bfb7db2e58c02dd68c8d584ba639fe4308704,wallet,wallet,1,94,wallet-wallet,1
29174,0xec001d0000004536cad29291f4000000d029abb2,wallet,wallet,1,73,wallet-wallet,1
15710,0x7d6a729a3f8055198696f5ac7587b4e086033151,wallet,wallet,1,60,wallet-wallet,1


In [116]:
fig_test = px.histogram(pdf_txn_to_address_diversity[(pdf_txn_to_address_diversity['unique_txns_bin']==1)
                            & (pdf_txn_to_address_diversity['from_to_contract']=='wallet-wallet')],
        x='total_txns')
fig_test.show()

In [26]:
pdf_txn_to_address_diversity[(pdf_txn_to_address_diversity['unique_txns_bin']==1)
                            & (pdf_txn_to_address_diversity['from_to_contract']=='wallet-wallet')].sort_values('total_txns',ascending=False).head()
                            # pdf_txn_to_address_diversity[(pdf_txn_to_address_diversity['unique_txns_bin']==1)
#                             & (pdf_txn_to_address_diversity['from_to_contract']=='wallet-wallet')].sort_values('total_txns',ascending=False).shape[0]
# 8003

,from,from_contract,to_contract,unique_txns,total_txns,from_to_contract,unique_txns_bin
26684,0xd7c09e006a2891880331b0f6224071c1e890a98a,wallet,wallet,1,157,wallet-wallet,1
7793,0x3d8793145fbeb2496a8fc33346da70b6ad2572dd,wallet,wallet,1,95,wallet-wallet,1
7178,0x384bfb7db2e58c02dd68c8d584ba639fe4308704,wallet,wallet,1,94,wallet-wallet,1
29174,0xec001d0000004536cad29291f4000000d029abb2,wallet,wallet,1,73,wallet-wallet,1
15710,0x7d6a729a3f8055198696f5ac7587b4e086033151,wallet,wallet,1,60,wallet-wallet,1


In [18]:
pdf_txn_address_diversity

,from_to_contract,unique_txns_bin,hist_unique_txns,feaquency_mean,feaquency_max,feaquency_std
0,contract-contract,1,259,16.980695,806,82.783431
1,contract-contract,2,126,13.865079,223,27.597494
2,contract-contract,3,74,78.662162,1742,252.837014
3,contract-contract,4,24,28.000000,113,29.554666
4,contract-contract,5,7,58.714286,163,55.670801
5,contract-contract,6,5,331.600000,684,274.745883
6,contract-contract,7,12,108.750000,185,47.926317
7,contract-contract,8,24,2449.291667,9707,2912.881398
8,contract-wallet,1,155,1.735484,30,2.932304
9,contract-wallet,2,49,3.244898,16,2.681189


In [29]:
fig = px.bar(pdf_txn_address_diversity[pdf_txn_address_diversity['from_to_contract']=='contract-contract'].sort_values('unique_txns_bin'),
x='unique_txns_bin',y=['hist_unique_txns','feaquency_mean','feaquency_max','feaquency_std'],#,color='from_to_contract',
barmode='group',text_auto=True,title= 'Transaction diversity -- frequency of unique txns -- stats -- contract-contract')
fig.show()

In [21]:
fig = px.bar(pdf_txn_address_diversity[pdf_txn_address_diversity['from_to_contract']=='contract-wallet'].sort_values('unique_txns_bin'),
x='unique_txns_bin',y=['hist_unique_txns','feaquency_mean','feaquency_max','feaquency_std'],#,color='from_to_contract',
barmode='group',text_auto=True, title= 'Transaction diversity -- frequency of unique txns -- stats -- contract-wallet')
fig.show()

In [22]:
fig = px.bar(pdf_txn_address_diversity[pdf_txn_address_diversity['from_to_contract']=='wallet-contract'].sort_values('unique_txns_bin'),
x='unique_txns_bin',y=['hist_unique_txns','feaquency_mean','feaquency_max','feaquency_std'],#,color='from_to_contract',
barmode='group',text_auto=True, title= 'Transaction diversity -- frequency of unique txns -- stats -- wallet-contract')
fig.show()

In [23]:
fig = px.bar(pdf_txn_address_diversity[pdf_txn_address_diversity['from_to_contract']=='wallet-contract'].sort_values('unique_txns_bin'),
x='unique_txns_bin',y=['hist_unique_txns','feaquency_mean','feaquency_max','feaquency_std'],#,color='from_to_contract',
barmode='group',text_auto=True, title= 'Transaction diversity -- frequency of unique txns -- stats -- wallet-wallet')
fig.show()

In [85]:
fig = px.bar(pdf_txn_address_diversity.sort_values('unique_txns_bin'),
x='unique_txns_bin',y=['mean_feaquency'],color='from_to_contract',
barmode='group',text_auto=True)
fig.show()

In [104]:
pdf_txn_to_address_diversity_seg = pdf_transactions_wallet.groupby(['from'],as_index= False)['to'].agg({'unique_count': "nunique"})

pdf_txn_to_address_diversity_seg1 = pdf_txn_to_address_diversity_seg[pdf_txn_to_address_diversity_seg['unique_count']<=10]
pdf_txn_to_address_diversity_seg2 = pdf_txn_to_address_diversity_seg[pdf_txn_to_address_diversity_seg['unique_count']>10]
pdf_txn_to_address_diversity_seg1.shape[0], pdf_txn_to_address_diversity_seg2.shape[0]

(23839, 534)

## Analysis of sessions
- same value transaferred in close porximity A-B, B-C == A-C

# EDA on Holding data

In [10]:
pdf_holding.tail(10)

,blockNumber,timeStamp,value_usd,date_time,date,address,holding
840108,15345856,1660564522,-253105.10,2022-08-15 11:55:22,2022-08-15,0xdef1c0ded9bec7f1a1670819833240f027b25eff,0.000000e+00
840109,15345856,1660564522,-57838.75,2022-08-15 11:55:22,2022-08-15,0x8206412c107ef1adb70b9277974f5163760e128e,1.485357e+07
840110,15345857,1660564530,25247.15,2022-08-15 11:55:30,2022-08-15,0xe1573b9d29e2183b1af0e743dc2754979a40d237,3.163982e+07
840111,15345857,1660564530,17450.55,2022-08-15 11:55:30,2022-08-15,0xe1573b9d29e2183b1af0e743dc2754979a40d237,3.165727e+07
840112,15345857,1660564530,-25247.15,2022-08-15 11:55:30,2022-08-15,0xd249942f6d417cbfdcb792b1229353b66c790726,2.946085e+04
840113,15345857,1660564530,-17450.55,2022-08-15 11:55:30,2022-08-15,0x4cf0602a77a4243b768ee28aa52994ad5dfdbe46,7.704960e+04
840114,15345859,1660564560,14539.13,2022-08-15 11:56:00,2022-08-15,0xe1573b9d29e2183b1af0e743dc2754979a40d237,3.167181e+07
840115,15345859,1660564560,-14539.13,2022-08-15 11:56:00,2022-08-15,0x2d9ce245675640ad8e152fe2ab081d4cc375ff99,2.785408e+05
840116,15345860,1660564570,1674.47,2022-08-15 11:56:10,2022-08-15,0x8fb012d125264b9f2274fdaed105c26d6c0e772b,1.674470e+03
840117,15345860,1660564570,-1674.47,2022-08-15 11:56:10,2022-08-15,0x9a22cdb1ca1cdd2371cd5bb5199564c4e89465eb,6.098811e+05


In [11]:
pdf_holding[pdf_holding['address'] =='0x8206412c107ef1adb70b9277974f5163760e128e']

,blockNumber,timeStamp,value_usd,date_time,date,address,holding
689652,14776858,1652574892,8.47,2022-05-15 00:34:52,2022-05-15,0x8206412c107ef1adb70b9277974f5163760e128e,8.47
692734,14784288,1652676799,944.07,2022-05-16 04:53:19,2022-05-16,0x8206412c107ef1adb70b9277974f5163760e128e,952.54
692735,14784288,1652676799,7019337.02,2022-05-16 04:53:19,2022-05-16,0x8206412c107ef1adb70b9277974f5163760e128e,7020289.56
692740,14784290,1652676817,3765.41,2022-05-16 04:53:37,2022-05-16,0x8206412c107ef1adb70b9277974f5163760e128e,7024054.97
692741,14784290,1652676817,14117.06,2022-05-16 04:53:37,2022-05-16,0x8206412c107ef1adb70b9277974f5163760e128e,7038172.03
...,...,...,...,...,...,...,...
839850,15344919,1660551212,-6642.11,2022-08-15 08:13:32,2022-08-15,0x8206412c107ef1adb70b9277974f5163760e128e,15086566.42
839929,15345154,1660554263,-181441.97,2022-08-15 09:04:23,2022-08-15,0x8206412c107ef1adb70b9277974f5163760e128e,14905124.45
839931,15345155,1660554286,3140.69,2022-08-15 09:04:46,2022-08-15,0x8206412c107ef1adb70b9277974f5163760e128e,14908265.14
839935,15345156,1660554292,3142.02,2022-08-15 09:04:52,2022-08-15,0x8206412c107ef1adb70b9277974f5163760e128e,14911407.16


## Generated Date related info
- Holding per date
- take the last value of holding on each date
- gap between consecutive holding change dates

In [135]:
pdf_holding = pd.read_csv("../data_raw/frax_hist_holding.csv")
# convert datetime from date-str
fn_ts = lambda x: datetime.strptime(x, '%Y-%m-%d')
pdf_holding.date = pdf_holding.date.apply(fn_ts)

pdf_holding_perDate = pdf_holding.groupby(['address','date'], as_index=False)['holding'].agg({'holding_per_date':'last'})
# pdf_holding_perDate.shape (128468, 5)
# get the date difference between two consecutive holding entry, holdings get increased or decresead based on trasnactions
pdf_holding_perDate['date_shift_1'] = pdf_holding_perDate.groupby('address')['date'].shift(-1)
pdf_holding_perDate['cons_days_diff'] = (pdf_holding_perDate['date_shift_1']- pdf_holding_perDate['date']).dt.days
# replace days_diff of the last date by 1 since after  
# pdf_holding_perDate.fillna(1.0,inplace=True)

In [54]:
# original holding OK tested
pdf_holding[(pdf_holding['address'] =='0x8206412c107ef1adb70b9277974f5163760e128e') & (pdf_holding['date'] =='2022-05-16')].tail(5)

,blockNumber,timeStamp,value_usd,date_time,date,address,holding
694028,14787388,1652719494,-10491.30,2022-05-16 16:44:54,2022-05-16,0x8206412c107ef1adb70b9277974f5163760e128e,6907422.02
694214,14787927,1652726604,13053.14,2022-05-16 18:43:24,2022-05-16,0x8206412c107ef1adb70b9277974f5163760e128e,6920475.16
694241,14787984,1652727646,19776.96,2022-05-16 19:00:46,2022-05-16,0x8206412c107ef1adb70b9277974f5163760e128e,6940252.12
694653,14788493,1652734969,10371.20,2022-05-16 21:02:49,2022-05-16,0x8206412c107ef1adb70b9277974f5163760e128e,6950623.32
694734,14788609,1652736446,11487.14,2022-05-16 21:27:26,2022-05-16,0x8206412c107ef1adb70b9277974f5163760e128e,6962110.46


In [56]:
pdf_holding_perDate[pdf_holding_perDate['address'] =='0x8206412c107ef1adb70b9277974f5163760e128e'].head(5)


,address,date,holding_per_date,date_shift_1,cons_days_diff
64287,0x8206412c107ef1adb70b9277974f5163760e128e,2022-05-15,8.47,2022-05-16 00:00:00,1.0
64288,0x8206412c107ef1adb70b9277974f5163760e128e,2022-05-16,6962110.46,2022-05-17 00:00:00,1.0
64289,0x8206412c107ef1adb70b9277974f5163760e128e,2022-05-17,6959969.77,2022-05-18 00:00:00,1.0
64290,0x8206412c107ef1adb70b9277974f5163760e128e,2022-05-18,6596890.80,2022-05-19 00:00:00,1.0
64291,0x8206412c107ef1adb70b9277974f5163760e128e,2022-05-19,6404195.17,2022-05-20 00:00:00,1.0


## Analysis on Holding days
count    25741.000000
mean        65.847830
std        105.023629
min          0.000000
25%          0.000000
50%          9.000000
75%         94.000000
max        603.000000

In [73]:
# number of days holding
# get the max date and min date per address
pdf_holding_timegap = pdf_holding.groupby('address', as_index=False)['date'].agg({'holding_max_date':max,
                                                                                  'holding_min_date':min})
# join wallet-name info 
pdf_holding_timegap_2 = (pdf_holding_perDate.merge(pdf_holding_timegap,on=['address'], how='inner')
                                           .merge(pdf_address_name, on=['address'], how='inner'))
# pdf_holding_timegap.shape # (25741, 5)
# pdf_holding_timegap_2.shape # (25741, 5)
# address with holding span ==20 days
# pdf_holding_timegap_2[pdf_holding_timegap_2['holding_span']==20]
# '0x0bcf1aae86bf5b5fd706a0440d35e046685e76e1'
# '0xfe661f36d7dd32a4a647623e6890f9405f32d3bf'
# '0xff416d3e86ad027d806869ed17d4928b2a3aed4f'


In [76]:
pdf_holding_timegap_2[pdf_holding_timegap_2['address'] =='0x0bcf1aae86bf5b5fd706a0440d35e046685e76e1']

,address,date,holding_per_date,date_shift_1,cons_days_diff,holding_max_date,holding_min_date,name,contract,holding_span
7443,0x0bcf1aae86bf5b5fd706a0440d35e046685e76e1,2020-12-21,6.040000e+00,2020-12-23 00:00:00,2.0,2021-01-10,2020-12-21,NaN,wallet,20
7444,0x0bcf1aae86bf5b5fd706a0440d35e046685e76e1,2020-12-23,1.080151e+04,2020-12-27 00:00:00,4.0,2021-01-10,2020-12-21,NaN,wallet,20
7445,0x0bcf1aae86bf5b5fd706a0440d35e046685e76e1,2020-12-27,6.891080e+03,2020-12-30 00:00:00,3.0,2021-01-10,2020-12-21,NaN,wallet,20
7446,0x0bcf1aae86bf5b5fd706a0440d35e046685e76e1,2020-12-30,-7.275958e-12,2021-01-04 00:00:00,5.0,2021-01-10,2020-12-21,NaN,wallet,20
7447,0x0bcf1aae86bf5b5fd706a0440d35e046685e76e1,2021-01-04,5.032802e+04,2021-01-05 00:00:00,1.0,2021-01-10,2020-12-21,NaN,wallet,20
7448,0x0bcf1aae86bf5b5fd706a0440d35e046685e76e1,2021-01-05,3.482802e+04,2021-01-10 00:00:00,5.0,2021-01-10,2020-12-21,NaN,wallet,20
7449,0x0bcf1aae86bf5b5fd706a0440d35e046685e76e1,2021-01-10,-9.549694e-12,1.0,1.0,2021-01-10,2020-12-21,NaN,wallet,20


### Fix last Holding date
- if user holds more than an amount X after the holding-max-date that means the user is not transacting but holding data
- then the max-holding date becomes the last date on this table /download_date-('2022-08-15 00:00:00')
- find the amount X- threshold
- *Report:* 60 % of the population has less than 50 at the last date. So threshold: 50

In [120]:
pdf_holding_max_date = pdf_holding_timegap_2.copy()
# Primarily get the days difference                                                                                  
pdf_holding_max_date['holding_span']  = (pdf_holding_max_date['holding_max_date'] - pdf_holding_max_date['holding_min_date']).dt.days


In [116]:
# the case where the holding is >100 after the max holding date
# in this case max holding date should be the download_date as the user still holding the data
pdf_holding_max_date[(pdf_holding_max_date['date'] == pdf_holding_max_date['holding_max_date']) & (pdf_holding_max_date['holding_per_date']>100)]
# 2442 rows × 10 columns

,address,date,holding_per_date,date_shift_1,cons_days_diff,holding_max_date,holding_min_date,name,contract,holding_span
1445,0x0000000000e52081b4531278fcd274e5f3cd86b5,2022-01-07,82192.61,NaT,NaN,2022-01-07,2021-11-18,NaN,wallet,50
1607,0x000000000dfde7deaf24138722987c9a6991e2d4,2022-08-15,364106.29,NaT,NaN,2022-08-15,2022-04-03,NaN,wallet,134
2040,0x0000006daea1723962647b7e189d311d757fb793,2021-06-27,101.43,NaT,NaN,2021-06-27,2021-04-30,NaN,wallet,58
2066,0x000000d608918b0dc2fe0214b03dd620e8564e81,2022-03-14,696.28,NaT,NaN,2022-03-14,2022-03-14,NaN,wallet,0
2079,0x00000c3ee2b0cad91c32e42f3f29ef89efaeb042,2022-08-09,25414.84,NaT,NaN,2022-08-09,2022-07-26,NaN,wallet,14
...,...,...,...,...,...,...,...,...,...,...
128333,0xffcf0cdf83168376e2ef94e5d1c91a5cc1b5f8fc,2022-08-13,14331.41,NaT,NaN,2022-08-13,2022-08-10,NaN,wallet,3
128337,0xffd4dae0d7d8ddb6f408dca0a47763ae3a57f4ce,2022-06-17,27705.53,NaT,NaN,2022-06-17,2022-06-17,NaN,wallet,0
128402,0xffd70ed81bd9eefe8d0ef4cbbfafb40c234ff957,2022-07-25,49996.28,NaT,NaN,2022-07-25,2021-05-07,NaN,wallet,444
128426,0xffe4f909c9fc16b34d907b1f1c919c8f7f110423,2021-11-05,226.91,NaT,NaN,2021-11-05,2021-11-05,NaN,wallet,0


In [40]:
fig = px.histogram(pdf_holding_max_date[(pdf_holding_max_date['date'] == pdf_holding_max_date['holding_max_date'])
                                          & (pdf_holding_max_date['holding_per_date'].between(1,1000))],
     x="holding_per_date", nbins=30, histnorm='percent',text_auto=True,color='contract',barmode="group",cumulative=True, range_x=["1","100"],
    title='holding amount on max-holding-date -- Histogram',width=800,height=500) #'histnorm='probability density') #, color='contract'
fig.update_layout(bargap=0.1)
fig.show()

# Threshold: 50

In [97]:
# the case where the holding is >50 after the max holding date
# in this case max holding date should be the download_date as the user still holding the data

# pdf_holding_max_date['holding_max_date_2'] = np.where((pdf_holding_max_date['date'] == pdf_holding_max_date['holding_max_date'])
#          & (pdf_holding_max_date['holding_per_date']>50),
#          '2022-08-15', str(pdf_holding_max_date['holding_max_date']))

# replace the max holding date
pdf_holding_max_date.loc[(pdf_holding_max_date['date'] == pdf_holding_max_date['holding_max_date'])
         & (pdf_holding_max_date['holding_per_date']>50),'holding_max_date'] = datetime.strptime('2022-08-15', '%Y-%m-%d')

# calculate the span again
pdf_holding_max_date['holding_span']  = (pdf_holding_max_date['holding_max_date'] - pdf_holding_max_date['holding_min_date']).dt.days

In [123]:
pdf_holding_max_date[(pdf_holding_max_date['date'] == pdf_holding_max_date['holding_max_date']) & (pdf_holding_max_date['holding_per_date']>50)]
# 76 rows × 10 columns Number of rows reduced OK Tested

,address,date,holding_per_date,date_shift_1,cons_days_diff,holding_max_date,holding_min_date,name,contract,holding_span
1607,0x000000000dfde7deaf24138722987c9a6991e2d4,2022-08-15,364106.29,NaT,NaN,2022-08-15,2022-04-03,NaN,wallet,134
2202,0x0000e0ca771e21bd00057f54a68c30d400000000,2022-08-15,3735.21,NaT,NaN,2022-08-15,2022-02-26,NaN,wallet,170
4563,0x04fc84d0b2914005cd16fa377ae889d768b2e7ff,2022-08-15,2166.83,NaT,NaN,2022-08-15,2021-12-23,UniswapV2Pair,contract,235
4619,0x0523928faac80cee3a0378526f9aa13b85dd62f7,2022-08-15,26220.98,NaT,NaN,2022-08-15,2022-01-07,NaN,wallet,220
7034,0x0a92ac70b5a187fb509947916a8f63dd31600f80,2022-08-15,924430.13,NaT,NaN,2022-08-15,2022-05-28,FraxswapPair,contract,79
...,...,...,...,...,...,...,...,...,...,...
123498,0xf653c1c4d825c1600932ef1cc4db1d755e91ed05,2022-08-15,1269.85,NaT,NaN,2022-08-15,2021-11-03,NaN,wallet,285
123882,0xf6d78d9c50ffbc2917e374fd185b8ff89c55effc,2022-08-15,408.60,NaT,NaN,2022-08-15,2022-05-27,NaN,wallet,80
124397,0xf79f7834261b875d8c958b632e3223129b2432da,2022-08-15,6351.20,NaT,NaN,2022-08-15,2021-11-12,NaN,wallet,276
127359,0xfd0a40bc83c5fae4203dec7e5929b446b07d1c76,2022-08-15,25085.05,NaT,NaN,2022-08-15,2020-12-20,UniswapV2Pair,contract,603


In [128]:
# save intermediate data
# pdf_holding_max_date.to_csv('../data_checks/frax_hist_eda_holding_stag2_1.csv',index=False)
# pdf_holding_max_date.shape (128468, 10)

### distribution of Holding dates

In [131]:
pdf_holding_2 = pd.read_csv('../data_checks/frax_hist_eda_holding_stag2_1.csv')
# pdf_holding_max_date.shape (128468, 10) OK!

In [134]:
pdf_holding_2.tail()

,address,date,holding_per_date,date_shift_1,cons_days_diff,holding_max_date,holding_min_date,name,contract,holding_span
128463,0xfffceb4d636d3d151ae4fea25126281fd28f3f5d,2021-02-21,0.0,NaN,NaN,2021-02-21,2021-02-21,NaN,wallet,0
128464,0xfffda0296fa8b2830fdfe8daf4f8e5996b679e87,2021-02-10,5000.0,2021-02-11,1.0,2021-04-02,2021-02-10,NaN,wallet,51
128465,0xfffda0296fa8b2830fdfe8daf4f8e5996b679e87,2021-02-11,60000.0,2021-02-13,2.0,2021-04-02,2021-02-10,NaN,wallet,51
128466,0xfffda0296fa8b2830fdfe8daf4f8e5996b679e87,2021-02-13,0.0,2021-04-02,48.0,2021-04-02,2021-02-10,NaN,wallet,51
128467,0xfffda0296fa8b2830fdfe8daf4f8e5996b679e87,2021-04-02,0.0,NaN,NaN,2021-04-02,2021-02-10,NaN,wallet,51


In [187]:
pdf_holding_span = pdf_holding_2.groupby(['address','contract'],as_index=False)['holding_span'].agg({'holding_span':'first'})

In [188]:
pdf_holding_span.tail()

,address,contract,holding_span
25736,0xfff87808e95bc582286ff711d8a1340933f6c447,wallet,0
25737,0xfffa285b4ca6ead9e537279e6b0d6d233d5e56bb,wallet,34
25738,0xfffa6a40d645369736fdef9a529e3e5b744fc21e,wallet,278
25739,0xfffceb4d636d3d151ae4fea25126281fd28f3f5d,wallet,0
25740,0xfffda0296fa8b2830fdfe8daf4f8e5996b679e87,wallet,51


In [183]:
pdf_holding_span.holding_span.describe()

count    25741.000000
mean        72.999495
std        110.095230
min          0.000000
25%          0.000000
50%         15.000000
75%        110.000000
max        603.000000
Name: holding_span, dtype: float64

In [184]:
fig = px.histogram(pdf_holding_span, x="holding_span", nbins=10, histnorm='percent',text_auto=True,
    title='holding days-span histogram',width=800,height=500) #'histnorm='probability density') #, color='contract'
fig.update_layout(bargap=0.1)
fig.show()

In [185]:
fig = px.histogram(pdf_holding_span, x="holding_span", nbins=10, histnorm='percent',text_auto=True,cumulative=True,
    title='holding days-span histogram',width=800,height=500) #'histnorm='probability density') #, color='contract'
fig.update_layout(bargap=0.1)
fig.show()

In [189]:
fig = px.histogram(pdf_holding_span, x="holding_span", nbins=10, histnorm='percent',text_auto=True,
    color='contract',barmode="group",
    title='holding days-span grouped by contract/wallet histogram',width=800,height=500) #'histnorm='probability density') #, color='contract'
fig.update_layout(bargap=0.1)
fig.show()

In [190]:
fig = px.histogram(pdf_holding_span, x="holding_span", nbins=10, histnorm='percent',text_auto=True,
    color='contract',barmode="group",cumulative=True,
    title='holding days-span grouped by contract/wallet histogram',width=800,height=500) #'histnorm='probability density') #, color='contract'
fig.update_layout(bargap=0.1)
fig.show()

## Holding Amount

In [ ]:
pdf_holding_2['date'] = pd.to_datetime(pdf_holding_2['date'])
pdf_holding_2['date_shift_1'] = pd.to_datetime(pdf_holding_2['date_shift_1'])
pdf_holding_2['holding_max_date'] = pd.to_datetime(pdf_holding_2['holding_max_date'])
pdf_holding_2['holding_min_date'] = pd.to_datetime(pdf_holding_2['holding_min_date'])
# pdf_holding_2.dtypes

In [163]:
pdf_holding_2.tail()

,address,date,holding_per_date,date_shift_1,cons_days_diff,holding_max_date,holding_min_date,name,contract,holding_span
128463,0xfffceb4d636d3d151ae4fea25126281fd28f3f5d,2021-02-21,0.0,2021-02-22,1,2021-02-21,2021-02-21,NaN,wallet,0
128464,0xfffda0296fa8b2830fdfe8daf4f8e5996b679e87,2021-02-10,5000.0,2021-02-11,1,2021-04-02,2021-02-10,NaN,wallet,51
128465,0xfffda0296fa8b2830fdfe8daf4f8e5996b679e87,2021-02-11,60000.0,2021-02-13,2,2021-04-02,2021-02-10,NaN,wallet,51
128466,0xfffda0296fa8b2830fdfe8daf4f8e5996b679e87,2021-02-13,0.0,2021-04-02,48,2021-04-02,2021-02-10,NaN,wallet,51
128467,0xfffda0296fa8b2830fdfe8daf4f8e5996b679e87,2021-04-02,0.0,2021-04-03,1,2021-04-02,2021-02-10,NaN,wallet,51


In [158]:
# replace the nulls of dateshif_1 by the max date 
# nulls appear as when we shift by 1 the last date has no shift-1 date available
# by replacing with max-holding-date we want to calculate the continuus holding span
pdf_holding_2['date_shift_1'].fillna(pdf_holding_2['holding_max_date'] + pd.DateOffset(1), inplace=True)
pdf_holding_2['cons_days_diff'] = (pdf_holding_2['date_shift_1']- pdf_holding_2['date']).dt.days

In [164]:
# calculate date weighted holding amount
pdf_holding_2['days_weighted_holding'] = pdf_holding_2['holding_per_date'] * pdf_holding_2['cons_days_diff']

In [165]:
pdf_holding_2[pdf_holding_2['address'] =='0x0bcf1aae86bf5b5fd706a0440d35e046685e76e1']

,address,date,holding_per_date,date_shift_1,cons_days_diff,holding_max_date,holding_min_date,name,contract,holding_span,days_weighted_holding
7443,0x0bcf1aae86bf5b5fd706a0440d35e046685e76e1,2020-12-21,6.040000e+00,2020-12-23,2,2021-01-10,2020-12-21,NaN,wallet,20,1.208000e+01
7444,0x0bcf1aae86bf5b5fd706a0440d35e046685e76e1,2020-12-23,1.080151e+04,2020-12-27,4,2021-01-10,2020-12-21,NaN,wallet,20,4.320604e+04
7445,0x0bcf1aae86bf5b5fd706a0440d35e046685e76e1,2020-12-27,6.891080e+03,2020-12-30,3,2021-01-10,2020-12-21,NaN,wallet,20,2.067324e+04
7446,0x0bcf1aae86bf5b5fd706a0440d35e046685e76e1,2020-12-30,-7.275958e-12,2021-01-04,5,2021-01-10,2020-12-21,NaN,wallet,20,-3.637979e-11
7447,0x0bcf1aae86bf5b5fd706a0440d35e046685e76e1,2021-01-04,5.032802e+04,2021-01-05,1,2021-01-10,2020-12-21,NaN,wallet,20,5.032802e+04
7448,0x0bcf1aae86bf5b5fd706a0440d35e046685e76e1,2021-01-05,3.482802e+04,2021-01-10,5,2021-01-10,2020-12-21,NaN,wallet,20,1.741401e+05
7449,0x0bcf1aae86bf5b5fd706a0440d35e046685e76e1,2021-01-10,-9.549694e-12,2021-01-11,1,2021-01-10,2020-12-21,NaN,wallet,20,-9.549694e-12


In [171]:
pdf_holding_amnt = pdf_holding_2.groupby(['address','holding_span','contract'],as_index=False)['days_weighted_holding'].agg({'sum_days_weighted_holding': sum})
pdf_holding_amnt['day_weighted_avg_holding'] = round(pdf_holding_amnt['sum_days_weighted_holding'] / pdf_holding_amnt['holding_span'], 2 )

In [172]:
pdf_holding_amnt[pdf_holding_amnt['address'] =='0x0bcf1aae86bf5b5fd706a0440d35e046685e76e1']

,address,holding_span,contract,sum_days_weighted_holding,day_weighted_avg_holding
1396,0x0bcf1aae86bf5b5fd706a0440d35e046685e76e1,20,wallet,288359.48,14417.97


In [194]:
fig = px.histogram(pdf_holding_amnt[pdf_holding_amnt['day_weighted_avg_holding'].between(0, 25000)],
    x="day_weighted_avg_holding", nbins=20, histnorm='percent',text_auto=True,
    title='day_weighted_avg_holding -- Histogram',width=800,height=500, color='contract',barmode="group")#,cumulative=True,range_x=["-5000000","5000000"],
fig.update_layout(bargap=0.1)
fig.show()
# [pdf_holding_amnt['day_weighted_avg_holding'].between(-5000000, 5000000)]